In [35]:
import pandas as pd
import numpy as np
from myutils import 

In [36]:
url = "data/cars-dataset1.csv"
df = pd.read_csv(url)

In [37]:
df.head()

,Car Names,Mileages,Ratings,Reviews,used/certified,Price
0,2019 Honda Accord Sport,Mileage,3.7,(3 reviews),Used,"$27,995.00"
1,2015 Chevrolet Corvette Stingray Z51,"29,192 mi.",3.7,(3 reviews),Used,"$52,000.00"
2,2020 Jeep Grand Cherokee Overland,"53,561 mi.",4.8,"(1,147 reviews)",Used,"$41,998.00"
3,2019 Audi A7 3.0T Prestige,"46,664 mi.",4.9,"(1,831 reviews)",Used,"$62,750.00"
4,2017 Ford Mustang GT Premium,"21,726 mi.",4.0,(409 reviews),Used,"$29,995.00"


In [27]:
# initialize features for filtered dataset
new_df = {}
names = ['name', 'year', 'mileage', 'used/certified', 'rating', 'reviews']

for name in names:
    new_df[name] = []
    

# filter data and put into new data file (new_df)
for sample in df.values:
    
    # Some rows have mileage = 'Mileage'. We ignore those rows.
    if sample[1] == 'Mileage':
        continue
    
    new_df['name'].append(sample[0][5:])
    new_df['year'].append(int(sample[0][0:4]))
    new_df['mileage'].append()
    
    

In [28]:
new_df


{'name': ['Chevrolet Corvette Stingray Z51',
  'Jeep Grand Cherokee Overland',
  'Audi A7 3.0T Prestige',
  'Ford Mustang GT Premium',
  'Jeep Grand Cherokee Limited',
  'Jeep Grand Cherokee High Altitude',
  'Ford Fusion SE',
  'Ford F-150 Raptor',
  'GMC Yukon Denali',
  'Kia Telluride SX',
  'Jeep Wrangler Unlimited Sport',
  'Toyota 4Runner Limited',
  'Toyota Camry SE',
  'INFINITI Q50 3.0T Red Sport 400',
  'Ford F-150 Raptor',
  'Lexus GS 350 Base',
  'Mitsubishi Mirage ES',
  'Subaru Outback Touring',
  'Lexus ES 350 Base',
  'Toyota Land Cruiser Heritage Edition',
  'BMW M3 Base',
  'Toyota Avalon Limited',
  'Ford Mustang EcoBoost Premium',
  'Acura ILX Technology & A-Spec',
  'Subaru Outback 3.6R Touring',
  'Nissan Frontier S',
  'Lamborghini Urus Base',
  'Nissan Leaf SL',
  'Nissan Rogue SV',
  'Chevrolet Silverado 1500 LTZ',
  'GMC Yukon Denali',
  'Mitsubishi Mirage ES',
  'Acura MDX 3.5L',
  'Jeep Grand Cherokee Limited',
  'BMW X5 M Base',
  'Honda CR-V EX',
  'Ford E